In [1]:
import sys

# sys.path.append(r'/Users/laisky/repo/laisky/ramjet/ramjet/settings')
sys.path.append(r'/opt/configs/ramjet')
sys.path
import prd

In [9]:
from datetime import datetime
import traceback
import re

from pymongo import MongoClient
import tweepy
from tweepy import API, OAuthHandler


mongo = MongoClient("localhost", 27017)
tweets = mongo['twitter']['tweets']

auth = OAuthHandler(prd.CONSUMER_KEY, prd.CONSUMER_SECRET)
auth.set_access_token(prd.ACCESS_TOKEN, prd.ACCESS_TOKEN_SECRET)
api = API(auth, wait_on_rate_limit=True, parser=tweepy.parsers.JSONParser())
api.me()

{'id': 105351466,
 'id_str': '105351466',
 'name': 'Laisky',
 'screen_name': 'ppcelery',
 'location': '♥ Shanghai',
 'profile_location': None,
 'description': '热爱文学的理科生，本质是一台高语速大存储的复读机。',
 'url': 'https://t.co/tmLO9n4B1E',
 'entities': {'url': {'urls': [{'url': 'https://t.co/tmLO9n4B1E',
     'expanded_url': 'https://about.me/laisky',
     'display_url': 'about.me/laisky',
     'indices': [0, 23]}]},
  'description': {'urls': []}},
 'protected': False,
 'followers_count': 7544,
 'friends_count': 228,
 'listed_count': 133,
 'created_at': 'Sat Jan 16 03:02:39 +0000 2010',
 'favourites_count': 849,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'verified': False,
 'statuses_count': 21894,
 'lang': None,
 'status': {'created_at': 'Sat Jun 22 14:23:42 +0000 2019',
  'id': 1142438015064629249,
  'id_str': '1142438015064629249',
  'text': '@Steverezero 云备份。\nwinter is coming, it may delay, but never absent.',
  'truncated': False,
  'entities': {'hashtags': [],
   'symbols': 

In [4]:
api.user_timeline(count=1)

[{'created_at': 'Sat Jun 22 14:23:42 +0000 2019',
  'id': 1142438015064629249,
  'id_str': '1142438015064629249',
  'text': '@Steverezero 云备份。\nwinter is coming, it may delay, but never absent.',
  'truncated': False,
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [{'screen_name': 'Steverezero',
     'name': 'D-NARUTO',
     'id': 2246680200,
     'id_str': '2246680200',
     'indices': [0, 12]}],
   'urls': []},
  'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
  'in_reply_to_status_id': 1142437378176405504,
  'in_reply_to_status_id_str': '1142437378176405504',
  'in_reply_to_user_id': 2246680200,
  'in_reply_to_user_id_str': '2246680200',
  'in_reply_to_screen_name': 'Steverezero',
  'user': {'id': 105351466,
   'id_str': '105351466',
   'name': 'Laisky',
   'screen_name': 'ppcelery',
   'location': '♥ Shanghai',
   'description': '热爱文学的理科生，本质是一台高语速大存储的复读机。',
   'url': 'https://t.co/tmLO9n4B1E',
   'entities': {'url': {'urls'

In [57]:
# delete tweets


def is_status_tobe_delete(tweet):
    if len(tweet.get('entities', {}).get("hashtags", [])) >= 1:
        return False
    
    if tweet['created_at'] < datetime(2017,6,5):
        return True

def delete():
#     last_tweets = api.user_timeline(count=1)
#     if not last_tweets:
#         return
    
#     print(last_tweets[0].created_at < datetime(2019,7,1))
#     print(last_tweets[0])
#     print(dir(last_tweets[0]))
#     return

#     current_id = last_tweets[0].id
    current_id = "379207474345046017"
    n = n_del = 1
#         tweets = api.user_timeline(max_id=current_id, count=1000)
    for tweet in tweets.find({'user.screen_name': 'ppcelery'}).sort("_id", -1):
        n += 1

        if is_status_tobe_delete(tweet):
            n_del += 1
            print("delete", tweet['id'],tweet['created_at'])
            try:
                api.destroy_status(tweet['id'])
            except Exception:
                traceback.print_exc()
        else:
            current_id = tweet['id']

        if n % 1000 == 0:
            print(">> scan {} statuses".format(n-1))
        if n_del % 100 == 0:
            print(">> delete {} statuses".format(n_del-1))


delete()

>> scan 999 statuses
>> scan 1999 statuses
>> scan 2999 statuses
>> scan 3999 statuses
>> scan 4999 statuses
>> scan 5999 statuses
>> scan 6999 statuses
>> scan 7999 statuses
>> scan 8999 statuses
>> scan 9999 statuses
>> scan 10999 statuses
>> scan 11999 statuses
>> scan 12999 statuses
>> scan 13999 statuses
>> scan 14999 statuses
>> scan 15999 statuses
>> scan 16999 statuses
>> scan 17999 statuses
>> scan 18999 statuses
>> scan 19999 statuses
>> scan 20999 statuses
>> scan 21999 statuses
>> scan 22999 statuses
>> scan 23999 statuses
>> scan 24999 statuses
>> scan 25999 statuses
>> scan 26999 statuses
>> scan 27999 statuses


KeyboardInterrupt: 

In [67]:
# download image
import requests
from pathlib import Path

dirpath = r'/var/www/uploads/twitter'
# dirpath = r'/Users/laisky/Downloads'


def download_images_for_tweet(tweet):
    for img in tweet['entities']['media']:
        with requests.get(img['media_url_https']+":orig") as r:
            if r.status_code != 200:
                print("got error", r.text)
                continue
                
            fpath = Path(dirpath, img['media_url_https'].split("/")[-1])
            if fpath.is_file():
                continue
                
            with open(fpath, 'wb') as f:
                f.write(r.content)    
            print("tweet img ok", tweet['id'], fpath)
            

def download_images():
    for tweet in tweets.find({"entities.media": {"$exists": 1}}).sort("_id", -1):
        download_images_for_tweet(tweet)
        
        
download_images()
        

tweet img ok 719103735004200961 /Users/laisky/Downloads/CfrFCRcUEAIFaEe.jpg
tweet img ok 719468584750096384 /Users/laisky/Downloads/CfwQ3VxVIAAj1Yv.jpg
got error 


UnboundLocalError: local variable 'fpath' referenced before assignment

In [11]:
# download related tweets

def twitter_api_parser(tweet):
    reg_topic = re.compile(r"[\b|\s]#(\S+)")
    tweet['topics'] = reg_topic.findall(
        tweet['text'].replace('.', '_')
    )
    tweet['created_at'] = datetime.strptime(
        tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y'
    )

    # replace url
    t = tweet['text']
    if 't.co' in t:
        # parse entities media
        if 'media' in tweet['entities']:
            for media in tweet['entities']['media']:
                surl = media['url']
                eurl = media['media_url']
                t = t.replace(surl, eurl)

        # parse entities urls
        if 'urls' in tweet['entities']:
            for d in tweet['entities']['urls']:
                surl = d['url']
                eurl = d['expanded_url']
                t = t.replace(surl, eurl)

        tweet['text'] = t

    return tweet


def save_relate_tweets(status):
    related_ids = []
    status.get("in_reply_to_status_id") and related_ids.append(status['in_reply_to_status_id'])
    status.get("retweeted_status") and related_ids.append(status['retweeted_status']['id'])
    status.get("quoted_status") and related_ids.append(status['quoted_status']['id'])
    related_ids = filter(lambda id_: not tweets.find_one({"id": id_}), related_ids)
    
    for id_ in related_ids:
        try:
            docu = api.get_status(id_)
        except Exception:
            print(f"load tweet {id_} got error")
        else:
            print(f"save tweet [{docu['user']['screen_name']}]{docu['id']}")
            save_tweet(docu)
            save_relate_tweets(docu)
            
def save_tweet(docu):
    docu = twitter_api_parser(docu)
    tweets.update_one(
        {'id': docu['id']},
        {'$set': docu},
        upsert=True
    )

def download_relate_tweets():
    for tweet in tweets.find(no_cursor_timeout=True):
        save_relate_tweets(tweet)

        
download_relate_tweets()


save tweet 482450130839142400
save tweet 314018018000777216


KeyboardInterrupt: 